In [ ]:
# ------------------------------------------------------------------
# Example: memory-efficient statistics on a large NetCDF
# ------------------------------------------------------------------
# What “chunks” means:
#   Xarray can read the file in small blocks (chunks) instead of all
#   at once.  Only the blocks you actually need are pulled into RAM,
#   so even a laptop can handle a 30-m grid for all of Ontario. Read
#   more at: docs.xarray.dev/en/stable/generated/xarray.DataArray.chunk.html
# ------------------------------------------------------------------

In [ ]:
#Import xarray (handles NetCDF)
import xarray as xr

In [ ]:
#Open the dataset and tell xarray to use automatic chunk sizes
#    (file must be saved in ../Data/)
ds = xr.open_dataset(
    "../Data/Ontario_30m_Manure_FW_N_P_kg_ha.nc",
    chunks="auto"        # read the file in small blocks on demand
)

In [ ]:
#Choose the variable we care about: total manure N (kg/ha/yr)
da = ds["total_manure_N"]

In [ ]:
# Full-grid statistics
# Count how many cells are >0 across the entire province
non_zero_full = (da > 0).sum().compute()  # compute() pulls only counts
total_full    = da.size
coverage_full = float(non_zero_full) / total_full * 100
print(f"Non-zero coverage – full grid: {coverage_full:.2f}%")

In [ ]:
# Provincial average of the same variable
mean_full = da.mean().compute().item()
print(f"Ontario-wide mean: {mean_full:.2f} kg/ha/yr")

In [ ]:
# Windsor → Ottawa window statistics
xmin, xmax = -9_700_000, -8_000_000
ymin, ymax =  5_050_000,  5_780_000

# Extract that rectangle
da_win = da.sel(
    lon_5070=slice(xmin, xmax),
    lat_5070=slice(ymin, ymax)
)

# Count non-zero cells inside the window
non_zero_win = (da_win > 0).sum().compute()
total_win    = da_win.size
coverage_win = float(non_zero_win) / total_win * 100
print(f"Non-zero coverage – window:    {coverage_win:.2f}%")